In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from scipy.signal import find_peaks, butter, filtfilt

# Random Seed
tf.random.set_seed(6950)
np.random.seed(6950)

# Constants
SAMPLING_RATE = 500  # Hz
ECG_FOLDER = "../../../Datasets/12-lead electrocardiogram database/ECGData"
DIAGNOSTICS_FILE = "../../../Datasets/12-lead electrocardiogram database/Diagnostics.xlsx"

# Rhythm Mapping
RHYTHM_MAPPING = {
    'AFIB': 'AFIB',
    'AF': 'AFIB',
    'SVT': 'GSVT',
    'AT': 'GSVT',
    'SAAWR': 'GSVT',
    'ST': 'GSVT',
    'AVNRT': 'GSVT',
    'AVRT': 'GSVT',
    'SB': 'SB',
    'SR': 'SR',
    'SA': 'SR'
}

In [4]:
# Hamilton-Tompkins QRS Detection
def preprocess_ecg(ecg_signal, sampling_rate=SAMPLING_RATE):
    """Preprocess ECG data: Bandpass filter and detect R-peaks."""
    # Bandpass filter (0.5 - 50 Hz)
    lowcut = 0.5
    highcut = 50.0
    nyquist = 0.5 * sampling_rate
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(1, [low, high], btype='band')
    filtered_ecg = filtfilt(b, a, ecg_signal)

    # Derivative
    derivative = np.diff(filtered_ecg)

    # Squaring
    squared = derivative ** 2

    # Moving window integration
    window_size = int(0.150 * sampling_rate)  # 150ms
    integrated = np.convolve(squared, np.ones(window_size) / window_size, mode='same')

    # Find R-peaks
    r_peaks, _ = find_peaks(integrated, distance=sampling_rate * 0.6, height=np.mean(integrated))
    return r_peaks, filtered_ecg


def extract_features(ecg_data, r_peaks):
    """Extract features from ECG data and R-peaks."""
    rr_intervals = np.diff(r_peaks) / SAMPLING_RATE * 1000  # in ms
    ventricular_rate = 60000 / np.mean(rr_intervals) if len(rr_intervals) > 0 else 0
    qrs_count = len(r_peaks)
    return {
        "VentricularRate": ventricular_rate,
        "QRSCount": qrs_count
    }


# Load and preprocess data
diagnostics_df = pd.read_excel(DIAGNOSTICS_FILE)
diagnostics_df['MappedRhythm'] = diagnostics_df['Rhythm'].map(RHYTHM_MAPPING)

features = []
labels = []

for idx, row in diagnostics_df.iterrows():
    try:
        ecg_file = os.path.join(ECG_FOLDER, row['FileName'] + ".csv")
        if os.path.exists(ecg_file):
            ecg_data = pd.read_csv(ecg_file, skiprows=1, usecols=[1]).to_numpy().flatten()
            r_peaks, filtered_ecg = preprocess_ecg(ecg_data)
            feature = extract_features(ecg_data, r_peaks)
            features.append(feature)
            labels.append(row['MappedRhythm'])
    except Exception as e:
        print(f"Error processing file {row['FileName']}: {e}")

# Convert to DataFrame
features_df = pd.DataFrame(features)
features_df['Label'] = labels

# Split data
X = features_df.drop('Label', axis=1).values
y = features_df['Label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6950, stratify=y)

# Normalize features for MLP
X_train_norm = (X_train - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)
X_test_norm = (X_test - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)

# Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=6950)
dt_classifier.fit(X_train, y_train)
dt_preds = dt_classifier.predict(X_test)
print("Decision Tree Classification Report:")
print(classification_report(y_test, dt_preds, digits=5))

# MLP Classifier
mlp_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(np.unique(y_train)), activation='softmax')
])

mlp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mlp_model.fit(X_train_norm, y_train, epochs=50, batch_size=32, validation_split=0.2)

mlp_preds = np.argmax(mlp_model.predict(X_test_norm), axis=1)
print("MLP Classification Report:")
print(classification_report(y_test, mlp_preds, digits=5))

Decision Tree Classification Report:
              precision    recall  f1-score   support

        AFIB    0.30325   0.37753   0.33634       445
        GSVT    0.30568   0.30303   0.30435       462
          SB    0.80970   0.83676   0.82301       778
          SR    0.46497   0.32809   0.38472       445

    accuracy                        0.51878      2130
   macro avg    0.47090   0.46135   0.46210      2130
weighted avg    0.52255   0.51878   0.51727      2130

Epoch 1/50


/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1735025423.763817  587681 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1735025423.797674  587681 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1735025423.

InvalidArgumentError: Graph execution error:

Detected at node data_1 defined at (most recent call last):
<stack traces unavailable>
Detected at node data_1 defined at (most recent call last):
<stack traces unavailable>
Detected unsupported operations when trying to compile graph __inference_one_step_on_data_1214[] on XLA_GPU_JIT: _Arg (No registered '_Arg' OpKernel for XLA_GPU_JIT devices compatible with node {{node data_1}}
	 (OpKernel was found, but attributes didn't match) Requested Attributes: T=DT_STRING, _output_shapes=[[32]], _user_specified_name="data", index=1){{node data_1}}
The op is created at: 
File "home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/runpy.py", line 196, in _run_module_as_main
File "home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/runpy.py", line 86, in _run_code
File "home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
File "home/denuvo